In [1]:
import numpy as np 
import os
import sys
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import datetime
from scipy.interpolate import griddata

proj_dir = os.path.join(os.pardir,os.pardir)
data_dir = os.path.join(proj_dir,'data','preprocessing')
src_dir = os.path.join(proj_dir,'src')

sys.path.append(src_dir)
from tools.log_progress import log_progress

# equid dummy bry with lat, lon and depth coordinates

In [6]:
bry_path = os.path.join(data_dir,'processed','sd_bry.nc')
sd_bry = xr.open_dataset(bry_path)
sd_bry

<xarray.Dataset>
Dimensions:     (eta_rho: 205, eta_v: 204, one: 1, s_rho: 31, salt_time: 12, temp_time: 12, v2d_time: 12, v3d_time: 12, xi_rho: 257, xi_u: 256, zeta_time: 12)
Coordinates:
  * temp_time   (temp_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
  * salt_time   (salt_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
  * v3d_time    (v3d_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 197.7 ...
  * v2d_time    (v2d_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 197.7 ...
  * zeta_time   (zeta_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
Dimensions without coordinates: eta_rho, eta_v, one, s_rho, xi_rho, xi_u
Data variables:
    theta_s     (one) float64 ...
    theta_b     (one) float64 ...
    Tcline      (one) float64 ...
    hc          (one) float64 ...
    sc_r        (s_rho) float64 ...
    Cs_r        (s_rho) float64 ...
    temp_north  (temp_time, s_rho, xi_rho) float64 ...
    salt_north  (salt_time, s_rho, xi_rho) float64 ...
    u_north   

In [3]:
sd_grd_path = os.path.join(data_dir,'processed','sd_grd.nc')
sd_grd = xr.open_dataset(sd_grd_path)

In [4]:
from tools.calc_z import calc_z
sd_grd['depth_rho'] = xr.DataArray(calc_z(sd_grd.h,sd_grd.zice,7,8,20,31)[0],dims=['s_rho','eta_rho','xi_rho'])
depths = calc_z(sd_grd.h,sd_grd.zice,7,8,20,31)[0]
depths_u = (depths[:,:,1:]+depths[:,:,:-1])/2
depths_v = (depths[:,1:,:]+depths[:,:-1,:])/2
sd_grd['depth_u'] = xr.DataArray(depths_u,dims=['s_rho','eta_u','xi_u'])
sd_grd['depth_v'] = xr.DataArray(depths_v,dims=['s_rho','eta_v','xi_v'])

In [7]:
for key in ['rho','u','v']:
    sd_bry['lat_'+key+'_south']=sd_grd['lon_'+key][0,:]
    sd_bry['lon_'+key+'_south']=sd_grd['lon_'+key][0,:]
    sd_bry['depth_'+key+'_south']=sd_grd['depth_'+key][0,:]
    
    sd_bry['lat_'+key+'_north']=sd_grd['lon_'+key][-1,:]
    sd_bry['lon_'+key+'_north']=sd_grd['lon_'+key][-1,:]
    sd_bry['depth_'+key+'_north']=sd_grd['depth_'+key][-1,:]
        
    sd_bry['lat_'+key+'_east']=sd_grd['lon_'+key][:,-1]
    sd_bry['lon_'+key+'_east']=sd_grd['lon_'+key][:,-1]
    sd_bry['depth_'+key+'_east']=sd_grd['depth_'+key][:,-1]
    
    sd_bry['lat_'+key+'_west']=sd_grd['lon_'+key][:,0]
    sd_bry['lon_'+key+'_west']=sd_grd['lon_'+key][:,0]
    sd_bry['depth_'+key+'_west']=sd_grd['depth_'+key][:,0]
    

# load atom grd and data, select sd area and equip with depth coordinates

In [9]:
from dask.distributed import Client
c = Client()
c

Client Scheduler: tcp://127.0.0.1:37421 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 101.38 GB


In [20]:
atom_grd_path = os.path.join(data_dir,'processed','waom2_grd.nc')
atom_sd_sel = {'xi_rho':slice(2590,2770),'eta_rho':slice(1530,1675),
            'xi_u':slice(2590,2769),'eta_u':slice(1530,1675),
            'xi_v':slice(2590,2770),'eta_v':slice(1530,1674)}
atom_sd_sel_rho = {'xi_rho':slice(2590,2770),'eta_rho':slice(1530,1675)}
atom_grd = xr.open_dataset(atom_grd_path).isel(atom_sd_sel)

atom_avg_path = os.path.join(data_dir,os.pardir,'analysis','raw','waom2','ocean_avg_0538-0610_zeta.nc')
    
atom_avg = []    
atom_avg = xr.open_mfdataset(atom_avg_path).isel(atom_sd_sel_rho)

In [11]:
depths = calc_z(atom_grd.h,atom_grd.zice,7,8,250,31)[0]
depths_u = (depths[:,:,1:]+depths[:,:,:-1])/2
depths_v = (depths[:,1:,:]+depths[:,:-1,:])/2
atom_grd['depth_rho'] = xr.DataArray(depths,dims=['s_rho','eta_rho','xi_rho'])

atom_grd['depth_u'] = xr.DataArray(depths_u,dims=['s_rho','eta_u','xi_u'])
atom_grd['depth_v'] = xr.DataArray(depths_v,dims=['s_rho','eta_v','xi_v'])

In [69]:
sort = atom_avg['ocean_time.dayofyear']
atom_avg_sorted = atom_avg.sortby(sort)

In [70]:
atom_avg_sorted

<xarray.Dataset>
Dimensions:     (eta_rho: 145, ocean_time: 73, xi_rho: 180)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2015-01-01T12:00:00 ...
    x_rho       (eta_rho, xi_rho) float64 dask.array<shape=(145, 180), chunksize=(145, 180)>
    y_rho       (eta_rho, xi_rho) float64 dask.array<shape=(145, 180), chunksize=(145, 180)>
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    zeta        (ocean_time, eta_rho, xi_rho) float32 dask.array<shape=(73, 145, 180), chunksize=(73, 145, 180)>
Attributes:
    file:              ocean_avg_0538.nc
    format:            netCDF-3 64bit offset file
    Conventions:       CF-1.4
    type:              ROMS/TOMS nonlinear model averages file
    title:             Whole Antarctic and Ocean Application, 2 km resolution
    rst_file:          ocean_rst.nc
    his_base:          ocean_his
    avg_base:          ocean_avg
    grd_file:          /g/data2/gh9/oxr581/waom_frc/waom2_grd.nc
    ini_file:          ocean_rst.nc
  

In [35]:
import pandas as pd
target_times = pd.date_range('2007-01-03 12:00:00','2008',freq='5D')

In [53]:
sd_bry

<xarray.Dataset>
Dimensions:          (eta_rho: 205, eta_u: 205, eta_v: 204, one: 1, s_rho: 31, salt_time: 12, temp_time: 12, v2d_time: 12, v3d_time: 12, xi_rho: 257, xi_u: 256, xi_v: 257, zeta_time: 12)
Coordinates:
  * temp_time        (temp_time) datetime64[ns] 2007-01-03T12:00:00 ...
  * salt_time        (salt_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
  * v3d_time         (v3d_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
  * v2d_time         (v2d_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
  * zeta_time        (zeta_time) float64 15.21 45.62 76.04 106.5 136.9 167.3 ...
Dimensions without coordinates: eta_rho, eta_u, eta_v, one, s_rho, xi_rho, xi_u, xi_v
Data variables:
    theta_s          (one) float64 ...
    theta_b          (one) float64 ...
    Tcline           (one) float64 ...
    hc               (one) float64 ...
    sc_r             (s_rho) float64 ...
    Cs_r             (s_rho) float64 ...
    temp_north       (temp_time, s_rho, xi_rho) f

In [54]:
for time in ['temp_time','salt_time','v3d_time','v2d_time','zeta_time']:
    sd_bry[time] =  pd.date_range('2007-01-03 12:00:00','2008',periods=12)

In [63]:
zeta_east_dummy = sd_bry.zeta_east.resample(zeta_time='5D').interpolate('linear')

In [82]:
lat_s = atom_grd.lat_rho.values.flatten()
lon_s = atom_grd.lon_rho.values.flatten()

lon_t = sd_bry.lon_rho_north
lat_t = sd_bry.lat_rho_north

data_s = atom_avg_sorted.zeta.isel(ocean_time=0).values.flatten()
out = griddata((lat_s,lon_s),data_s,(lat_t,lon_t),'nearest')

array([-1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370181,
       -1.6370181, -1.6370181, -1.6370181, -1.6370181, -1.6370